In [1]:
import json
import numpy as np
from qiskit.circuit import QuantumCircuit,QuantumRegister,ClassicalRegister
from qiskit.circuit import ParameterVector
from qiskit_aer import Aer
from qiskit import transpile
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.quantum_info import SparsePauliOp,Operator, state_fidelity, Statevector
import random
from scipy.optimize import minimize

# 2 qubit entanglement detection
In this notebook I will se how we can detect 2 qubit entanglement using VQAs or similar procedures.

First we open the 4000 random unitaries form the json

In [2]:
with open(r"C:\Users\aquim\OneDrive\Desktop\Master\TFM\random_unitaries_2q.json", "r") as f:
    loaded_data = json.load(f)

# Convert back to NumPy arrays
loaded_matrices = [
    np.array(data["real"]) + 1j * np.array(data["imag"])
    for data in loaded_data
]
print(len(loaded_matrices))

4000


We have now stored the 4000 complex matrices in the loaded_matrices vector.
With this 4000 unitaries we will create a set of 2000 training states and 2000 test states.


In [3]:
training = loaded_matrices[0:2000]
test = loaded_matrices[2000:4000]

In [6]:
len(training)

2000

## Geometric measure of entanglement 

The first method we will review is the entanglement detection through the geometric measure (GME). This is a monotone which provides a quantification of how close this state is to the closest possible separable state. This closeness is calculated using the fideleity:
$$F(|\psi\rangle , |\phi\rangle) = |\langle \psi | \phi \rangle|^2 $$
We de define GME as:
$$E(|\psi \rangle) = 1 - \Lambda_{\psi}^2$$
Where the $\Lambda_{\psi}$ is defined as:
$$\Lambda_{\psi}^2 = max_{|\phi\rangle \in SEP} F(|\psi\rangle , |\phi\rangle) $$
If E is zero we have that the quantum state is separable, otherwise it is entangled.

By maximazing the fidelity, we try to obtain the closest separable state to the one we want to classify. It is worth noting that this procedure is possible because the set of separable states is a convex set.

In [10]:
backend = Aer.get_backend('statevector_simulator')

For the cost function, we calculate the fidelity the two circuits by using the fidelity function provided by qiskit.

In [11]:
def cost_fun(params,ansatz1,ansatz2,backend):
    ansatz2 = ansatz2.assign_parameters(params)
    isa_ansatz = transpile(ansatz1,backend=backend)
    isa_ansatz2 = transpile(ansatz2,backend=backend)
    sv1 = Statevector(isa_ansatz)
    sv2 = Statevector(isa_ansatz2)
    fid = state_fidelity(sv1,sv2)
    return -fid

For the benchmarking of this method, I define 3 quantum variational circuits:
1. ansatz_sep: variational quantum circuit with 6 parameters. I selected this ansatz because the 3 possible rotations (rx,ry,rz) consider every single state in the Bloch Sphere for each qubit. There is no entangling gates, which makes this ansatz separable.
2. qc: the first named qc quantum circuit creates the entangled state using the unitaries from the random unitary training dataset.
3. qc: the second qc quantum circuit creates a separable state using the unitaries from the traininf dataset.

In [100]:
ansatz_sep = QuantumCircuit(2)
ansatz_sep.ry(params[0],0)
ansatz_sep.rx(params[1],0)
ansatz_sep.ry(params[2],1)
ansatz_sep.rx(params[3],1)
ansatz_sep.rz(params[4],0)
ansatz_sep.rz(params[5],1)
real_label= []
q = 0
geom_entr = []
for i in range(0,int(len(training)/4),2):
    print(f'State {q}')
    q+=1
    rand_num = np.random.rand()
    # We select randomely if the generated state is entangled or not
    if rand_num <= 0.5:   
        # Entangled state generator
        real_label.append(1)
        x0 = np.random.rand(6) * 2 * np.pi
        qc =QuantumCircuit(2)
        qc.h(0)# We first create the  Bell state
        qc.cx(0,1)
        U_ = np.kron(training[i],training[i+1])# We do the tensor product and create the entangled state
        qc.unitary(Operator(U_), [0,1], label="U1⊗U2")
        qc.barrier()
    elif rand_num>0.5:
        # Separated state
        real_label.append(0)
        x0 = np.random.rand(6) * 2 * np.pi
        qc =QuantumCircuit(2)
        U_ = np.kron(training[0],training[1])# We do the tensor product and create the entangled state
        qc.unitary(Operator(U_), [0,1], label="U1⊗U2")
        qc.barrier()
    res = minimize(cost_fun,x0,args = (qc,ansatz_sep,backend),method='cobyla',tol=0.1)
    geom_entr.append(-res['fun'])
    

State 0
State 1
State 2
State 3
State 4
State 5
State 6
State 7
State 8
State 9
State 10
State 11
State 12
State 13
State 14
State 15
State 16
State 17
State 18
State 19
State 20
State 21
State 22
State 23
State 24
State 25
State 26
State 27
State 28
State 29
State 30
State 31
State 32
State 33
State 34
State 35
State 36
State 37
State 38
State 39
State 40
State 41
State 42
State 43
State 44
State 45
State 46
State 47
State 48
State 49
State 50
State 51
State 52
State 53
State 54
State 55
State 56
State 57
State 58
State 59
State 60
State 61
State 62
State 63
State 64
State 65
State 66
State 67
State 68
State 69
State 70
State 71
State 72
State 73
State 74
State 75
State 76
State 77
State 78
State 79
State 80
State 81
State 82
State 83
State 84
State 85
State 86
State 87
State 88
State 89
State 90
State 91
State 92
State 93
State 94
State 95
State 96
State 97
State 98
State 99
State 100
State 101
State 102
State 103
State 104
State 105
State 106
State 107
State 108
State 109
State 110


In [103]:
E = [(1 - p**2) for p in geom_entr]
for i in range(len(E)):
    # I set a bound at 0.1 for classification of separable and entangled
    if E[i] < 0.1:
        E[i]=0
    elif E[i]>=0.1:
        E[i]=1
q= 0
for z in range(len(real_label)):
    if E[z]==real_label[z]:
        q+=1
print(f'The method`s accuaracy is {(q/len(real_label))*100}%')

The method`s accuaracy is 100.0%
